In [1]:
import pandas as pd
import numpy as np
from llama_index.core import Document, SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.settings import Settings
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.llms.llama_cpp import LlamaCPP
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding    
import faiss

c:\Users\abhij\miniconda3\envs\RAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("./pubmed_abstracts/pubmed_abstracts.csv")

In [10]:
df = df.dropna(subset=["abstract"])

In [8]:
def dataframe_to_documents(df):
    docs = []
    for _, row in df.iterrows():
        text = row["abstract"]
        metadata = {
            "title": row.get("title", ""),
            "year": row.get("year", ""),
        }
        docs.append(Document(text=text, metadata=metadata))
    return docs

In [11]:
docs = dataframe_to_documents(df)

In [18]:
model_directory = r"E:\RAG_Models"
model_name = "phi-2-orange.Q4_K_M.gguf"

model_path = model_directory+"\\"+model_name

In [27]:
llm = LlamaCPP(model_path = model_path, temperature = 0.2, max_new_tokens = 256, context_window = 2048, verbose=True)

llama_model_loader: loaded meta data with 21 key-value pairs and 325 tensors from E:\RAG_Models\phi-2-orange.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi2
llama_model_loader: - kv   1:                               general.name str              = Phi2
llama_model_loader: - kv   2:                        phi2.context_length u32              = 2048
llama_model_loader: - kv   3:                      phi2.embedding_length u32              = 2560
llama_model_loader: - kv   4:                   phi2.feed_forward_length u32              = 10240
llama_model_loader: - kv   5:                           phi2.block_count u32              = 32
llama_model_loader: - kv   6:                  phi2.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi2.attention.head_count_kv u32  

In [28]:
Settings.llm = llm

In [37]:
Settings.embed_model = HuggingFaceEmbedding(model_name = "pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")

c:\Users\abhij\miniconda3\envs\RAG\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abhij\AppData\Local\llama_index\models--pritamdeka--BioBERT-mnli-snli-scinli-scitail-mednli-stsb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [39]:
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=50)
Settings.node_parser = text_splitter

In [40]:
dimensions = len(Settings.embed_model.get_text_embedding("sample text"))

faiss_index = faiss.IndexFlatL2(dimensions)
faiss_db = FaissVectorStore(faiss_index = faiss_index)
storage_context = StorageContext.from_defaults(vector_store = faiss_db)

In [41]:
index = VectorStoreIndex.from_documents(docs, storage_context=storage_context)

In [42]:
memory = ChatMemoryBuffer.from_defaults(token_limit = 600)
query_engine = index.as_query_engine(similarity_top_k = 3)

In [43]:
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine = query_engine,
    memory = memory
)

In [44]:
query = 'Is elevated dopamine consistently observed in patients with schizophrenia?'

In [45]:
response = chat_engine.chat(query)
print(response)

llama_perf_context_print:        load time =   31476.98 ms
llama_perf_context_print: prompt eval time =   31389.17 ms /  1090 tokens (   28.80 ms per token,    34.73 tokens per second)
llama_perf_context_print:        eval time =   49319.13 ms /   255 runs   (  193.41 ms per token,     5.17 tokens per second)
llama_perf_context_print:       total time =   81638.62 ms /  1345 tokens



Based on the provided studies, there is evidence of elevated dopamine in patients with schizophrenia, particularly in midbrain regions that project to parts of the striatum. However, the relationship between dopamine and schizophrenia is complex, and the findings are not consistent across all studies. Some studies have found preliminary evidence of an association between chronic social stress exposure and increased striatal dopamine functioning, but the quality of available studies is generally low. Therefore, while there is some evidence of elevated dopamine in schizophrenia, the findings are not universally consistent.

The studies also suggest that NM levels are higher in patients with schizophrenia than in healthy control individuals, which further supports the link between dopamine and schizophrenia. However, it's important to note that these findings are based on specific measures of dopamine and NM, and more research is needed to fully understand the relationship between dopami

In [ ]:
response.text